In [19]:
# L'objectif de ce notebook est de montrer mes capacités à "scrapper" des données présentes sur Internet et de les rendre utilisables. 
# Attention ! Ces données ne sont pas utilisées à des fins personnelles mais uniquement à titre de démonstration. 

# Ci-dessous les conditions d'utilisation des données du site

# 4. Propriété intellectuelle et contrefaçons.
# www.bien-dans-ma-ville.fr est propriétaire des droits de propriété intellectuelle et détient les droits d’usage sur tous les éléments accessibles sur le site internet, 
# notamment les textes, images, graphismes, logos, vidéos, icônes et sons. 
# Toute reproduction, représentation, modification, publication, adaptation de tout ou partie des éléments du site, 
# quel que soit le moyen ou le procédé utilisé, est interdite, sauf autorisation écrite préalable de : www.bien-dans-ma-ville.fr.
# Toute exploitation non autorisée du site ou de l’un quelconque des éléments qu’il contient sera considérée comme constitutive d’une contrefaçon et 
# poursuivie conformément aux dispositions des articles L.335-2 et suivants du Code de Propriété Intellectuelle. 

# Etape 1 - Import des librairies

import pandas as pd  
import numpy as np 

import requests 
from bs4 import BeautifulSoup 
import time 
import random 
# !pip install fake-useragent
from fake_useragent import FakeUserAgent 


In [20]:
# Gestion des paramètres 

# Création de headers différents 
ua = FakeUserAgent() 
useragent = ua.random 
print(useragent)

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20130401 Firefox/31.0


In [21]:
# Proxies 
proxies = []
response = requests.get('https://vpnhack.com/premium-proxy-list')
soup = BeautifulSoup(response.text, 'lxml')
for element in soup.find('tbody').find_all('tr'):
    ip = element.find('td').text
    port = element.findAll('td')[1].text
    port = port.replace(', 3128','')
    proxies.append(ip+':'+port)

proxies

['103.129.223.199:8080',
 '103.129.222.206:8080',
 '128.199.150.230:8080',
 '128.199.150.240:8080',
 '128.199.150.242:8080',
 '167.99.77.25:8080',
 '167.71.215.199:8080']

In [22]:
# Récupération d'un proxy de manière randomisée 

prox = random.choice(proxies)
prox

'167.71.215.199:8080'

In [23]:
# Récupération de la soup 

def get_soup(link, useragent, proxy):

    # Nous utilisons un proxy choisi au hasard parmi ceux de notre liste et un "useragent" différent également à chaque requête
    response = requests.get(link, proxies={'http':proxy} ,headers = {'headers' : useragent}, timeout =10)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


In [29]:
# Récupération des pages 
# Exemple d'adresse de page : 'https://www.bien-dans-ma-ville.fr/classement-ville-global/?page=1', 'https://www.bien-dans-ma-ville.fr/classement-ville-global/?page=2'
# Les pages ont toutes une base identique, il suffit juste de modifier le dernier chiffre en indiquant le numéro de la page. 

def get_pages(nb):
    # 21 pages max
    pages = []
    for i in range(1,nb+1):
        t = 'https://www.bien-dans-ma-ville.fr/classement-ville-global/?page='+str(i)
        pages.append(t)
    return pages

pages = get_pages(3)
pages

['https://www.bien-dans-ma-ville.fr/classement-ville-global/?page=1',
 'https://www.bien-dans-ma-ville.fr/classement-ville-global/?page=2',
 'https://www.bien-dans-ma-ville.fr/classement-ville-global/?page=3']

In [32]:
# Une fois nos pages récupérées, il nous faut extraire l'ensemble des liens vers les différentes villes.
# Une page contenant environ 30 villes 
# Pour se faire, nous allons extraire, à l'aide de BeautifulSoup et des balises HTML, les liens présents sur les pages.

links = []

def get_links(token):
    
    # A chaque itération la fonction requests.get() va essayer de se connecter à une adresse et enregister le résultat
    # A chaque itération, l'useragent ainsi que le proxy change. Cela permet d'éviter les blocages.
    response = requests.get(token, proxies = {'http':prox}, headers = {'headers':useragent}, timeout =5)

    soup = BeautifulSoup(response.text, 'lxml')
    for element in soup.find_all('td'):
        for n in element.find_all('a', href=True):
            links.append(n['href'])

get_links(pages[0])
# Les trois premiers résultats de notre liste de liens pour la première page (pages[0])
links[:3] 

['https://www.bien-dans-ma-ville.fr/tournefeuille-31557/avis.html',
 'https://www.bien-dans-ma-ville.fr/poisy-74213/avis.html',
 'https://www.bien-dans-ma-ville.fr/saint-mande-94067/avis.html']

In [26]:
# Une fois tous les liens obtenus, nous allons récupérer plusieurs informations disponibles parmi lesquelles:
# le nom de la ville, le nb d'habitants, le taux de pop active, le taux de chômage, les notes, le nbre de notes...
# Ces données seront ensuite 

cols = []
data = []

def get_data(token):

    # Tout d'abord, nous récupérons les données sur la page principale où se situe les informations concernant le nbre d'habitants, taux de chômage...
    # Pour se faire, nous devons retirer de notre lien 'avis.html'
    token = token.replace('avis.html', '')

    soup = get_soup(token, prox, useragent)
    data.append(soup.find('h1').find(text=True))

    for element in soup.find_all('table', {'class':'bloc_chiffre'}):
        for n in element.find_all('td')[0::3]:
            cols.append(n.text)
        for n in element.find_all('td')[1::3]:
            data.append(n.text)

    # Nous remettons la partie "avis.html"
    # Puis nous récupérons nos données 
    soup = get_soup(token+'avis.html', prox, useragent)

    for element in soup.find_all('table', {'class':'bloc_chiffre'}):
        for v in element.findAll('td')[::3]:
            cols.append(v.text)
        for n in element.findAll('td')[1::3]:
            data.append(n.text)

    cols.append('Nb Avis')

    for element in soup.find_all('div', {'class':'bloc_noterepartition'}):
        data.append(element.find('h3').text.split()[2])

In [27]:
# Lancement de notre projet 


links = []
df = pd.DataFrame()

#--- On récupére l'ensemble des liens ---
pages = get_pages(20)
for page in pages:
    get_links(page)
time.sleep(3)


# Pour chaque lien, nous récupérons les données 
for element in links[:100]:
    # A chaque itération, nous réinitialisons les listes de colonnes et data et ajoutons les données à notre dataframe.
    cols = ['Ville']
    data = []
    get_data(element)
    df = df.append([data])

df.columns = cols

df


,Ville,Nb habitants,Superficie,Pop densité,Pop active,Taux chômage,Revenu moyen,Prix m² moyen,Sécurité,Éducation,Sport / Loisir,Environnement,Vie pratique,Nb Avis
0,Tournefeuille,26 477,18 km²,1 471 h/km²,49.9%,5.3%,27 326 €/an,2 949 €,4.4,4.6,4.7,4.4,4.4,10
0,Poisy,7 594,11 km²,690 h/km²,51.1%,4%,26 169 €/an,3 719 €,4.5,4.5,4.5,4.4,4.5,11
0,Saint-Mandé,22 401,1 km²,22 401 h/km²,50.6%,5%,31 559 €/an,8 507 €,4.5,4.3,4.2,4.7,4.6,19
0,Erstein,10 969,36 km²,305 h/km²,51.8%,6%,21 426 €/an,NC,4.3,4.4,4.4,4.3,4.4,19
0,Vaison-la-Romaine,6 072,27 km²,225 h/km²,40.3%,6.9%,18 457 €/an,2 211 €,4.3,4.4,4.5,4.4,4.2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Suresnes,48 565,4 km²,12 141 h/km²,54.8%,5.6%,28 212 €/an,6 827 €,3.8,3.9,4.0,3.7,3.9,26
0,Maizières-lès-Metz,11 200,9 km²,1 244 h/km²,48.1%,7%,19 314 €/an,NC,3.6,4.1,3.8,3.8,4.0,12
0,Hénin-Beaumont,26 379,21 km²,1 256 h/km²,43.4%,8.6%,16 836 €/an,1 288 €,4.0,3.7,4.0,3.3,4.2,21
0,Olivet,21 639,23 km²,941 h/km²,46.4%,4.3%,25 415 €/an,2 234 €,3.5,3.7,4.0,3.8,4.2,10
